In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110650  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,2.057773e+07
2018-02-28,1.722802e+07
2018-03-31,2.011859e+07
2018-04-30,1.974328e+07
2018-05-31,1.979056e+07
2018-06-30,1.877333e+07
2018-07-31,2.014631e+07
2018-08-31,1.984132e+07
2018-09-30,1.861965e+07


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    2.057773e+07
 2018-02-28    1.722802e+07
 2018-03-31    2.011859e+07
 2018-04-30    1.974328e+07
 2018-05-31    1.979056e+07
 2018-06-30    1.877333e+07
 2018-07-31    2.014631e+07
 2018-08-31    1.984132e+07
 2018-09-30    1.861965e+07
 2018-10-31    2.015876e+07
 2018-11-30    1.975037e+07
 2018-12-31    1.993034e+07
 2019-01-31    2.030766e+07
 2019-02-28    1.757468e+07
 2019-03-31    2.014345e+07
 2019-04-30    1.947035e+07
 2019-05-31    1.967578e+07
 2019-06-30    1.866460e+07
 2019-07-31    1.999601e+07
 2019-08-31    1.949710e+07
 2019-09-30    1.794796e+07
 2019-10-31    1.949412e+07
 2019-11-30    1.987864e+07
 2019-12-31    2.025156e+07
 2020-01-31    1.963421e+07
 2020-02-29    1.584337e+07
 2020-03-31    1.533878e+07
 2020-04-30    1.455485e+07
 2020-05-31    1.585276e+07
 2020-06-30    1.648989e+07
 2020-07-31    1.777743e+07
 2020-08-31    1.780530e+07
 2020-09-30    1.553667e+07
 2020-10-31    1.748379e+07
 2020-11-30    1.878855e+07

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.248534
p-value : 0.017342
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1287.115, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1282.366, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1282.726, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1285.202, Time=0.09 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1288.119, Time=0.13 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1289.038, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1281.066, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1282.160, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1284.254, Time=0.11 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1285.354, Time=0.30 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1282.202, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1284.153, Time=0.20 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1286.177, Time=0.29 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.47 sec
 ARIMA(3,1,0)(0,0,0)

ARIMA(order=(1, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,1,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(1, 1, 0)   Log Likelihood                -639.948
Method:                       css-mle   S.D. of innovations        1452050.112
Date:                Sat, 11 Sep 2021   AIC                           1285.895
Time:                        02:09:58   BIC                           1291.036
Sample:                             1   HQIC                          1287.767
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.804e+04   1.78e+05     -0.383      0.702   -4.16e+05     2.8e+05
ar.L1.D.y     -0.2880      0.157     -1.831      0.067      -0.596       0.020
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([17741655.15740804, 17667395.38140839]), array([1452050.1117171 , 1782500.96807638]), array([[14895689.23469517, 20587621.08012091],
       [14173757.68157091, 21161033.08124587]]))
